# 导包

In [1]:
import pandas as pd

# 读取测试集

In [2]:
# 预测数据导入
data0 = pd.read_csv('../../数据/生成/预处理后测试数据.csv')
data0.head()

,serial_number,collect_time,1_hwerr_f,1_hwerr_e,2_hwerr_c,2_sel,3_hwerr_n,2_hwerr_s,3_hwerr_m,1_hwerr_st,...,3_hwerr_r,_hwerr_cd,3_sup_mce_note,3_cmci_sub,3_cmci_det,3_hwerr_pi,3_hwerr_o,3_hwerr_mce_l,manufacturer,vendor
0,server_23232,2019-06-14 02:18:30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,1.0
1,server_23232,2019-06-14 02:19:04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0
2,server_23232,2019-06-14 02:19:04,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0
3,server_35584,2019-06-14 07:36:01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0
4,server_35584,2019-06-14 07:36:07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0


In [3]:
# 筛选测试集特征
test = data0.drop(['serial_number','collect_time'],axis=1)

# 读取训练集

In [4]:
# 原始训练集数据
data = pd.read_csv('../../数据/生成/预处理后数据.csv')
# 制定分类训练数据
train0 = data.drop(['err_time','failure_time','collect_time','serial_number','tag'],axis=1)
# 制定回归训练数据
train1 = data.drop(['err','failure_time','collect_time','serial_number','tag'],axis=1)
train0.head()

C:\Users\76406\AppData\Local\Temp\ipykernel_15856\623381135.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../../数据/生成/预处理后数据.csv')


,1_hwerr_f,1_hwerr_e,2_hwerr_c,2_sel,3_hwerr_n,2_hwerr_s,3_hwerr_m,1_hwerr_st,1_hw_mem_c,3_hwerr_p,...,_hwerr_cd,3_sup_mce_note,3_cmci_sub,3_cmci_det,3_hwerr_pi,3_hwerr_o,3_hwerr_mce_l,manufacturer,vendor,err
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1


# 训练分类模型

In [5]:
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingRegressor

In [6]:
# 根据之前的测试结果选取模型
xgbc = XGBClassifier()

In [7]:
# 划分特征
tx = train0.iloc[:,:-1]
ty = train0.iloc[:,-1]

In [8]:
# 训练
xgbc.fit(tx, ty)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [9]:
# 对测试集预测
pre_y = xgbc.predict(test)
pre_y

array([0, 0, 1, ..., 0, 0, 0])

# 训练回归模型

In [10]:
# 划分特征
tx1 = train1.dropna(subset='err_time').iloc[:,:-1]
ty1 = train1.dropna(subset='err_time').iloc[:,-1]

In [11]:
# 根据之前的测试结果选取模型
gbr = GradientBoostingRegressor()

In [12]:
# 训练
gbr.fit(tx1,ty1)

GradientBoostingRegressor()

In [54]:
# 预测有故障机器的奔溃时间
pre_time = gbr.predict(test[pre_y==1]).tolist()
pre_time

[93.86171656457711,
 75.78285632272406,
 157.11284735364967,
 99.80135817602226,
 111.35261641483712,
 75.47960540716382,
 111.35261641483712,
 75.47960540716382,
 111.35261641483712,
 75.47960540716382,
 170.82454925214955,
 149.3467740710299,
 239.4384873517724,
 75.47960540716382,
 75.47960540716382,
 107.18231114513686,
 115.41850493393049,
 107.18231114513686,
 115.41850493393049,
 99.80135817602226,
 157.11284735364967,
 75.78285632272406,
 99.80135817602226,
 157.11284735364967,
 75.78285632272406,
 157.11284735364967,
 99.80135817602226,
 75.78285632272406,
 157.11284735364967,
 239.4384873517724,
 239.4384873517724,
 239.4384873517724,
 239.4384873517724,
 239.4384873517724,
 239.4384873517724,
 239.4384873517724,
 239.4384873517724,
 239.4384873517724,
 239.4384873517724,
 239.4384873517724,
 239.4384873517724,
 239.4384873517724,
 239.4384873517724,
 239.4384873517724,
 157.11284735364967,
 99.80135817602226,
 75.78285632272406,
 230.3357451025734,
 230.3357451025734,
 230.3

# 导出内存故障分析表

In [14]:
# 提取所有报错记录
pre_data = data0[pre_y == 1]

In [41]:
# 计算报错时间
pre_date = [pd.to_datetime(t) + pd.Timedelta(seconds=int(s)) for t,s in zip(pre_data['collect_time'],pre_time)]

In [66]:
# 根据比赛要求做表
pre_data = pd.DataFrame({'id':pre_data['serial_number'],'crash_time':pre_date,'crash_minutes':[round(i/60) for i in pre_time]})
pre_data

,id,crash_time,crash_minutes
2,server_23232,2019-06-14 02:20:37,2
1304,server_10807,2019-06-03 07:58:33,1
1306,server_10807,2019-06-03 07:59:55,3
1308,server_10807,2019-06-03 07:58:57,2
1663,server_47758,2019-06-10 13:31:14,2
...,...,...,...
127192,server_23,2019-06-15 01:24:25,12
127193,server_23,2019-06-15 01:25:45,13
127441,server_55420,2019-06-13 16:43:25,3
133855,server_33158,2019-06-13 16:35:54,3


In [76]:
# 筛选同编号下最快报错记录
oid = 'server_23232'
crash_minutes = 99999
may = 0
ind = []
for i,k,n in zip(pre_data['id'],pre_data['crash_minutes'],pre_data.index):
    if i != oid:
        oid = i
        crash_minutes = k
        ind.append(may)
        may = n
    else:
        if k < crash_minutes:
            crash_minutes = k
            may = n
ind

[2,
 1304,
 1664,
 1983,
 2001,
 3325,
 3572,
 3584,
 5677,
 5978,
 6600,
 7495,
 7693,
 9756,
 9776,
 10046,
 10412,
 10422,
 10547,
 10560,
 11193,
 11695,
 17322,
 21702,
 25692,
 26230,
 29496,
 35974,
 36101,
 41448,
 42416,
 42533,
 44623,
 44739,
 46737,
 47755,
 56712,
 58009,
 58509,
 62603,
 65137,
 66539,
 71321,
 71415,
 71665,
 75278,
 85173,
 111584,
 111915,
 112613,
 114728,
 115478,
 120686,
 124537,
 125725,
 127148,
 127441,
 133855]

In [83]:
# 导出
pre_data.loc[ind].to_csv('../../数据/生成/可能崩溃的机器信息.csv',index=False,header=False)